In [35]:
import pandas as pd
import json

# Load the technique-deception data
with open("deception_as_detection.json", "r") as f:
    technique_data = json.load(f)

# Extract Technique ID, Name, Tactic, and Deception Techniques
data = []
for technique_id, details in technique_data.items():
    technique_name = details["name"]
    tactic = (
        details["tactic"] if isinstance(details["tactic"], str) 
        else ", ".join(details["tactic"])  # Handle multiple tactics
    )
    deception_methods = ", ".join(details["deception_techniques"])  # Convert list to string

    # Store the extracted data
    data.append([technique_id, technique_name, tactic, deception_methods])

# Create a DataFrame
df_mapping = pd.DataFrame(data, columns=["Technique ID", "Technique Name", "Tactic", "Deception Techniques"])

# Save to CSV and display
df_mapping.to_csv("technique_deception_mapping_detection_as_deception.csv", index=False)
print("✅ Technique-Deception Mapping saved as technique_deception_mapping.csv")

# Show first few rows
df_mapping.head()


✅ Technique-Deception Mapping saved as technique_deception_mapping.csv


,Technique ID,Technique Name,Tactic,Deception Techniques
0,T1119,Automated Collection,Collection,Create fake directories and files (i.e. honeyf...
1,T1005,Data from Local System,Collection,Create fake directories and files (i.e. honeyf...
2,T1039,Data from Network Shared Drive,Collection,"Create fake network shares, or create fake dir..."
3,T1025,Data from Removable Media,Collection,Create decoy files or documents (beacons) that...
4,T1114,Email Collection,Collection,Create fake Outlook storage or cache files (.p...


In [36]:
df_mapping.columns

Index(['Technique ID', 'Technique Name', 'Tactic', 'Deception Techniques'], dtype='object')

In [37]:
import json
import pandas as pd

# Load JSON data (Replace 'data.json' with your JSON file path)
with open('d3fend_techniques.json', 'r') as file:
    data = json.load(file)

# Extract techniques data
techniques_list = []
for technique in data.get("techniques", []):
    technique_id = technique.get("techniqueID", "N/A")
    tactic = technique.get("tactic", "N/A")
    comment = technique.get("comment", "")

    # Extract deception technique from comment (assuming it's in parentheses)
    deception_technique = comment.split("(")[-1].split(")")[0] if "(" in comment and ")" in comment else "N/A"

    techniques_list.append([technique_id, tactic, deception_technique])

# Create DataFrame
df = pd.DataFrame(techniques_list, columns=["Technique ID", "Tactic", "Deception Techniques"])

# Save as CSV or Print Table
df.to_csv("d3fend_techniques_deception.csv", index=False)  # Save as CSV file
print(df)  # Print the table


    Technique ID                Tactic Deception Techniques
0          T1082             discovery     DecoyEnvironment
1          T1082             discovery   IntegratedHoneynet
2          T1082             discovery   StandaloneHoneynet
3      T1548.002  privilege-escalation            DecoyFile
4      T1548.002       defense-evasion            DecoyFile
..           ...                   ...                  ...
177        T1558     credential-access  DecoyUserCredential
178    T1558.001     credential-access  DecoyUserCredential
179        T1552     credential-access  DecoyUserCredential
180    T1550.001       defense-evasion  DecoyUserCredential
181    T1550.001      lateral-movement  DecoyUserCredential

[182 rows x 3 columns]


In [39]:
import pandas as pd
import requests

# Fetch MITRE ATT&CK techniques data
MITRE_ATTACK_DATA = requests.get('https://raw.githubusercontent.com/mitre/cti/master/enterprise-attack/enterprise-attack.json').json()

# Create a dictionary mapping technique ID to name
TECHNIQUES = {technique['external_references'][0]['external_id']: technique['name']
              for technique in MITRE_ATTACK_DATA['objects']
              if technique['type'] == 'attack-pattern' and not technique.get('revoked')}

# Function to get technique name based on ID
def get_technique(tid): 
    return TECHNIQUES.get(tid, "Unknown Technique") if '.' not in tid else "{}: {}".format(TECHNIQUES.get(tid[:5], "Unknown Parent"), TECHNIQUES.get(tid, "Unknown Sub-technique"))

# Load the CSV file
df = pd.read_csv("d3fend_techniques_deception.csv")

# Add a new column with technique names
df["Technique Name"] = df["Technique ID"].apply(get_technique)

# Save the updated CSV file
df.to_csv("d3fend_techniques_deception_updated.csv", index=False)

# Print sample output
print(df.head())


  Technique ID                Tactic Deception Techniques  \
0        T1082             discovery     DecoyEnvironment   
1        T1082             discovery   IntegratedHoneynet   
2        T1082             discovery   StandaloneHoneynet   
3    T1548.002  privilege-escalation            DecoyFile   
4    T1548.002       defense-evasion            DecoyFile   

                                      Technique Name  
0                       System Information Discovery  
1                       System Information Discovery  
2                       System Information Discovery  
3  Abuse Elevation Control Mechanism: Bypass User...  
4  Abuse Elevation Control Mechanism: Bypass User...  


In [47]:
import pandas as pd

# Load the two CSV files
df = pd.read_csv("technique_deception_mapping_detection_as_deception.csv")
df_mapping = pd.read_csv("d3fend_techniques_deception_updated.csv")

# Merge the two dataframes on all common columns using an outer join
df_merged = pd.merge(df, df_mapping, on=['Technique ID', 'Technique Name', 'Tactic', 'Deception Techniques'], how="outer")

# Save the merged table to a new CSV file
df_merged.to_csv("techniques_merged_table.csv", index=False)

# Print first few rows for verification
print(df_merged.head())


  Technique ID                                     Technique Name  \
0        T1003                                 Credential Dumping   
1    T1003.003                        OS Credential Dumping: NTDS   
2    T1003.005   OS Credential Dumping: Cached Domain Credentials   
3    T1003.007             OS Credential Dumping: Proc Filesystem   
4    T1003.008  OS Credential Dumping: /etc/passwd and /etc/sh...   

              Tactic                               Deception Techniques  
0  Credential Access  Inject fake credentials into LSASS (honey hash...  
1  credential-access                                DecoyUserCredential  
2  credential-access                                DecoyUserCredential  
3  credential-access                                          DecoyFile  
4  credential-access                                          DecoyFile  


In [28]:
df_mapping.columns

Index(['Technique ID', 'Technique Name', 'Tactic', 'Deception Techniques'], dtype='object')

In [33]:
print(df.columns)
print(df_mapping.columns)


Index(['Technique ID', 'Tactic', 'Deception Techniques'], dtype='object')
Index(['Technique ID', 'Technique Name', 'Tactic', 'Deception Techniques'], dtype='object')


In [6]:
pip install mitrecve

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13775 sha256=603e83d3654097702304d8aefcce5b5ef0281c65e3fa217c578122a51ef46a20
  Stored in directory: c:\users\anisa\appdata\local\pip\cache\wheels\1a\bf\a1\4cee4f7678c68c5875ca89eaccf460593539805c3906722228
Successfully built docopt
Note: you may need to restart the kernel to use updated packages.


In [20]:
import mitrecve

In [29]:
mitrecve.crawler.get_main_page("Domain Accounts Mappings")

{0: {'__PACKAGE': 'Domain Accounts Mappings',
  'ID': 'CVE-2025-26318',
  'URL': 'https://cve.mitre.org/cgi-bin/cvename.cgi?name=CVE-2025-26318',
  'DESC': 'Insecure permissions in TSplus Remote Access v17.30 allow attackers to retrieve a list of all domain accounts currently connected to the application.'},
 1: {'__PACKAGE': 'Domain Accounts Mappings',
  'ID': 'CVE-2025-25198',
  'URL': 'https://cve.mitre.org/cgi-bin/cvename.cgi?name=CVE-2025-25198',
  'DESC': "mailcow: dockerized is an open source groupware/email suite based on docker. Prior to version 2025-01a, a vulnerability in mailcow's password reset functionality allows an attacker to manipulate the `Host HTTP` header to generate a password reset link pointing to an attacker-controlled domain. This can lead to account takeover if a user clicks the poisoned link. Version 2025-01a contains a patch. As a workaround, deactivate the password reset functionality by clearing `Notification email sender` and `Notification email subject`